# Calcluate statstiics between products and data stest

In [1]:
# standard imports
import os
import scipy
import numpy as np
import xarray as xr
import pandas as pd

# machine learning libraries
import sklearn  # awesome machine-learning libary with many algorithms implemented
import xgboost as xgb    # extreme gradient boosting
import tensorflow as tf  # gold standard for neural-networks
from sklearn.ensemble import RandomForestRegressor # random forest regressor

# train test split
from sklearn.model_selection import train_test_split

# plotting
import cmocean as cm # nice colorbars
import matplotlib.pyplot as plt # for making any plots
# custom plotting I wrote
from plotting_tools.spatial_map import SpatialMap
from plotting_tools.time_series_diagram import TimeSeriesPlot

# regression metrics
from sklearn.metrics import r2_score
from sklearn.metrics import max_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
#from sklearn.metrics import explained_variance_score

# custom imports
import pgml

In [18]:
from scipy.stats import pearsonr

In [98]:
def bias(y_truth, y_pred):
    bias = y_pred.mean('time') - y_truth.mean('time')
    bias_mean = bias.mean()
    return bias_mean

def aae(y_truth, y_pred):
    aae = np.abs(y_pred - y_truth).mean('time')
    aae_mean = aae.mean()
    return aae_mean

def rmse(y_truth, y_pred):
    rmse = np.sqrt(mean_squared_error(y_truth, y_pred))
    return rmse

In [22]:
# hybrid model output
dir_output = '/local/data/artemis/workspace/gloege/LDEO-FFN/data/HPD_output/HPD_feat'

# hybrid file dictionary
dict_spco2 = {'CCSM':  f'{dir_output}/HPD-feat_CCSM_spco2_1x1_mon_198201-201712.nc',
              'CNRM':  f'{dir_output}/HPD-feat_CNRM_spco2_1x1_mon_198201-201712.nc',
              'MPI':   f'{dir_output}/HPD-feat_MPI_spco2_1x1_mon_198201-201712.nc',
              'NEMO':  f'{dir_output}/HPD-feat_NEMO_spco2_1x1_mon_198201-201712.nc',
              'NORESM':f'{dir_output}/HPD-feat_NORESM_spco2_1x1_mon_198201-201712.nc',   
              'RECOM': f'{dir_output}/HPD-feat_RECOM_spco2_1x1_mon_198201-201712.nc',
              'all':   f'{dir_output}/HPD-feat-all_spco2_1x1_mon_198201-201712.nc'
              }

# Model as target
#dir_hpd = '/local/data/artemis/workspace/gloege/LDEO-FFN/data/HPD_output/HPD_feat'
ds_hpd_feat = xr.open_mfdataset([dict_spco2[model] for model in ['CCSM', 'CNRM', 'MPI', 'NEMO', 'NORESM', 'RECOM']],
                               combine='nested', 
                               concat_dim='models').mean('models')
ds_hpd_feat = ds_hpd_feat.transpose('time', 'lat', 'lon')

In [161]:
ds_hpd_feat_all = xr.open_dataset(dict_spco2['all'])

In [146]:
dir_output = '/local/data/artemis/workspace/gloege/LDEO-FFN/data/HPD_output/HPD_target'

# hybrid file dictionary
dict_spco2_tar = {'CCSM':  f'{dir_output}/HPD-target_CCSM_spco2_1x1_mon_198201-201712.nc',
                  'CNRM':  f'{dir_output}/HPD-target_CNRM_spco2_1x1_mon_198201-201712.nc',
                  'MPI':    f'{dir_output}/HPD-target_MPI_spco2_1x1_mon_198201-201712.nc',
                  'NEMO':  f'{dir_output}/HPD-target_NEMO_spco2_1x1_mon_198201-201712.nc',
                  'NORESM':f'{dir_output}/HPD-target_NORESM_spco2_1x1_mon_198201-201712.nc',   
                  'RECOM': f'{dir_output}/HPD-target_RECOM_spco2_1x1_mon_198201-201712.nc',}

# Model as target
#dir_hpd = '/local/data/artemis/workspace/gloege/LDEO-FFN/data/HPD_output/HPD_feat'
ds_hpd_tar = xr.open_mfdataset([dict_spco2_tar[model] for model in dict_spco2_tar.keys()],
                               combine='nested', 
                               concat_dim='models').mean('models')
ds_hpd_tar = ds_hpd_tar.transpose('time', 'lat', 'lon')

In [3]:
mod_dir = '/local/data/artemis/observations/GCB_hindcast_models/processed/fgco2'
dict_fgco2 = {'CCSM':  f'{mod_dir}/CCSM-BEC_fgco2_1x1_198201-201712.nc',
              'CNRM':  f'{mod_dir}/CNRM-ESM2_fgco2_1x1_198201-201712.nc',
              'MPI':   f'{mod_dir}/MPI_fgco2_1x1_198201-201712.nc',
              'NEMO':  f'{mod_dir}/NEMO-planktom_fgco2_1x1_198201-201712.nc',
              'NORESM':f'{mod_dir}/NorESM_fgco2_1x1_198201-201712.nc',
              'RECOM': f'{mod_dir}/REcoM_jra_fgco2_1x1_198201-201712.nc',}



In [84]:
prod_dir = '/local/data/artemis/workspace/gloege/products/clean'

dict_prod = {'CSIR':  f'{prod_dir}/CSIR-ML6_1x1_198201-201612.nc', 
             'SOMFFN':  f'{prod_dir}/SOM-FFN_1x1_198201-201512.nc',
             'JMAMLR':  f'{prod_dir}/JMA-MLR_1x1_199001-201712.nc',  
             'LSCE':  f'{prod_dir}/FFNN-LSCE_1x1_198501-201812.nc',}

In [23]:
dir_tests = '/local/data/artemis/workspace/gloege/data/test_sets'

dict_data = {'glodap': f'{dir_tests}/GLODAP_1x1_198201-201812.nc',
            'carioca': f'{dir_tests}/CARIOCA_1x1_198201-201812.nc',
            'soccom': f'{dir_tests}/SOCCOM_1x1_198201-201812.nc',
            'ldeo': f'{dir_tests}/LDEO_1x1_198201-201812.nc'}

# Load time series

In [97]:
# time series 
dir_data = '/local/data/artemis/workspace/gloege'
# BATS
ds_bats = xr.open_dataset(f'{dir_data}/data/BATS/BATS.nc')

# HOT
ds_hot = xr.open_dataset(f'{dir_data}/data/HOT/HOT.nc')
ds_hot = ds_hot.where(ds_hot['spco2']>0)

# locations
BATS={'lat':31.83,
      'lon':360-(64.17)}

HOT={'lat':22.75,
      'lon':360-158}

ds_hot_mon = ds_hot.resample(time='1M').mean()


ds_hot_mon = ds_hot_mon.assign_coords(lon=(202.5),
                                      lat=(22.5)).expand_dims(['lat','lon'])
ds_hot_mon = ds_hot_mon.transpose('time','lat','lon')

### ================================================
### force time vector to be proper format and range
### ================================================
dates = pd.date_range(start='1988-10-01T00:00:00.000000000', 
                      end='2017-12-01T00:00:00.000000000',
                      freq='MS')+ np.timedelta64(14, 'D')
da_dates = xr.DataArray(dates, dims='time')
ds_hot_mon['time'] = dates
ds_hot_mon = ds_hot_mon.squeeze()



ds_bats_mon = ds_bats.resample(time='1M').mean()


ds_bats_mon = ds_bats_mon.assign_coords(lon=(295.5),
                                       lat=(31.5)).expand_dims(['lat','lon'])
ds_bats_mon = ds_bats_mon.transpose('time','lat','lon')


### ================================================
### force time vector to be proper format and range
### ================================================
dates = pd.date_range(start='1983-09-01T00:00:00.000000000', 
                      end='2018-08-01T00:00:00.000000000',
                      freq='MS')+ np.timedelta64(14, 'D')
da_dates = xr.DataArray(dates, dims='time')
ds_bats_mon['time'] = dates
ds_bats_mon = ds_bats_mon.squeeze()

/home/gloege/.conda/envs/tensorflow/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


# Calculate statistics between product and dataset

## For HPD

In [157]:
ds_data

<xarray.Dataset>
Dimensions:                    (lat: 180, lon: 360, time: 444)
Coordinates:
  * time                       (time) datetime64[ns] 1982-01-15 ... 2018-12-15
  * lat                        (lat) float64 -89.5 -88.5 -87.5 ... 88.5 89.5
  * lon                        (lon) float64 0.5 1.5 2.5 ... 357.5 358.5 359.5
Data variables:
    fCO2water_SST_wetatm_mean  (time, lat, lon) float64 ...
    fCO2water_SST_wetatm_std   (time, lat, lon) float64 ...

In [168]:
#for product in dict_spco2.keys():
#print(f'{product}')

ds_data = xr.open_dataset(dict_data['ldeo'])
ds_prod = ds_hpd_feat_all.compute()

(pred, obs) = xr.align(ds_prod['spco2'], ds_data['spco2_mean'])
pred = pred.where((obs>0) & (pred>0))
obs = obs.where((obs>0) & (pred>0))

txt= f'''
RMSE: {rmse(obs.stack(z=['time','lat','lon']).dropna('z').values, 
            pred.stack(z=['time','lat','lon']).dropna('z').values)}
bias: {bias(obs,pred).values}
corr: {pearsonr(obs.stack(z=['time','lat','lon']).dropna('z').values, 
                pred.stack(z=['time','lat','lon']).dropna('z').values)[0]}
AAE: {aae(obs,pred).values}

'''

print(txt)

/home/gloege/.conda/envs/tensorflow/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)



RMSE: 40.68455234134488
bias: -7.247976338073919
corr: 0.6330909051609723
AAE: 19.030151071734927




In [160]:
#for product in dict_prod.keys():
#print(f'{product}')

ds_data = ds_hot_mon['spco2']
ds_prod = ds_hpd_tar.compute().sel(lat=HOT['lat'], lon=HOT['lon'], method='nearest')

(pred, obs) = xr.align(ds_prod['spco2_hpd'], ds_data)
pred = pred.where((obs>0) & (pred>0))
obs = obs.where((obs>0) & (pred>0))

txt= f'''
RMSE: {rmse(obs.stack(z=['time']).dropna('z').values, 
            pred.stack(z=['time']).dropna('z').values)}
bias: {bias(obs,pred).values}
corr: {pearsonr(obs.stack(z=['time']).dropna('z').values, 
                pred.stack(z=['time']).dropna('z').values)[0]}
AAE: {aae(obs,pred).values}

'''

print(txt)

/home/gloege/.conda/envs/tensorflow/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)



RMSE: 9.411250799061813
bias: 4.587994294095665
corr: 0.9217669041526535
AAE: 7.18395057708797




## For other products

In [120]:
for product in dict_spco2.keys():
    print(f'{product}')
    
    ds_data = xr.open_dataset(dict_data['glodap'])
    ds_prod = xr.open_dataset(dict_spco2[product])

    (pred, obs) = xr.align(ds_prod['spco2'], ds_data['spco2_mean'])
    pred = pred.where((obs>0) & (pred>0))
    obs = obs.where((obs>0) & (pred>0))

    txt= f'''
    RMSE: {rmse(obs.stack(z=['time','lat','lon']).dropna('z').values, 
                pred.stack(z=['time','lat','lon']).dropna('z').values)}
    bias: {bias(obs,pred).values}
    corr: {pearsonr(obs.stack(z=['time','lat','lon']).dropna('z').values, 
                    pred.stack(z=['time','lat','lon']).dropna('z').values)[0]}
    AAE: {aae(obs,pred).values}

    '''

    print(txt)

CCSM


/home/gloege/.conda/envs/tensorflow/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)



    RMSE: 18.570389915531464
    bias: 1.408993890092329
    corr: 0.8642889306440947
    AAE: 12.486269437403662

    
CNRM


/home/gloege/.conda/envs/tensorflow/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)



    RMSE: 19.175652146724197
    bias: 2.0141024713494136
    corr: 0.8552956904426854
    AAE: 12.818712421652208

    
MPI


...


    RMSE: 19.582094534850143
    bias: 1.9965064935761379
    corr: 0.8509732868353663
    AAE: 12.864728880418129

    
RECOM


/home/gloege/.conda/envs/tensorflow/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)



    RMSE: 18.941855073334395
    bias: 1.4772536559534273
    corr: 0.8600981441171273
    AAE: 12.77044949685289

    
all


/home/gloege/.conda/envs/tensorflow/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)



    RMSE: 18.814846306984
    bias: 1.366469650604804
    corr: 0.8582423139501829
    AAE: 12.77706144109385

    


# Stats for time series

In [114]:
for product in dict_prod.keys():
    print(f'{product}')
    
    ds_data = ds_hot_mon['spco2']
    ds_prod = xr.open_dataset(dict_prod[product]).sel(lat=HOT['lat'],
                                                      lon=HOT['lon'], method='nearest')

    (pred, obs) = xr.align(ds_prod['spco2'], ds_data)
    pred = pred.where((obs>0) & (pred>0))
    obs = obs.where((obs>0) & (pred>0))

    txt= f'''
    RMSE: {rmse(obs.stack(z=['time']).dropna('z').values, 
                pred.stack(z=['time']).dropna('z').values)}
    bias: {bias(obs,pred).values}
    corr: {pearsonr(obs.stack(z=['time']).dropna('z').values, 
                    pred.stack(z=['time']).dropna('z').values)[0]}
    AAE: {aae(obs,pred).values}

    '''

    print(txt)

CSIR

    RMSE: 8.435497517667862
    bias: 1.668834314738831
    corr: 0.9055073891167784
    AAE: 6.7375705858633035

    
SOMFFN

    RMSE: 9.093575566160938
    bias: -0.04121513075500616
    corr: 0.889518429335657
    AAE: 7.098221593958732

    
JMAMLR

    RMSE: 11.272834136103333
    bias: 1.4520336051855907
    corr: 0.8791910129731604
    AAE: 8.851301024483037

    
LSCE

    RMSE: 9.52389658542815
    bias: 3.4997239299516423
    corr: 0.8974080903780008
    AAE: 7.461117781778249

    
